In [14]:
from typing import List, Tuple

def format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n".join([human, ai]) + "\n"
    return buffer

In [15]:

data = [('1', '2', 'a'), ('3', '4'), ('5', '6')]
print(format_chat_history(data))


Human: 1
Assistant: 2
Human: 3
Assistant: 4
Human: 5
Assistant: 6



## 네이버 API를 호출하여 검색하는 샘플

In [14]:
!pip install bs4
!pip install selenium
!pip install webdriver_manager

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 24.0 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)


In [ ]:
# 네이버 API를 호출하여 검색하는 샘플
import requests

def naver_search(query):
    client_id = "8IZ0G5iuDP3RfxlVCEX7"
    client_secret = "kj_oKc7ROo"
    url = "https://openapi.naver.com/v1/search/webkr.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret,
    }
    params = {"query": query, "display": 10}  # 최대 10개의 결과 표시
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # JSON 형식의 검색 결과
    else:
        return response.status_code

# 검색 예시
results = naver_search("13인치 노트북 추천")


In [17]:

# 검색 결과에서 링크 추출하고 내용 가져오기
from bs4 import BeautifulSoup

# def get_content_from_url(url: str) -> str:
#     """URL에서 본문 내용을 추출합니다"""
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             # HTML 파일 저장
#             filename = url.split('/')[-1]
#             if not filename.endswith('.html'):
#                 filename = f"{filename}.html"
#             with open(filename, 'w', encoding='utf-8') as f:
#                 f.write(response.text)
#             print(f"HTML 파일이 {filename}으로 저장되었습니다.")
            
#             # bs parsing            
#             soup = BeautifulSoup(response.text, 'html.parser')
            
#             # 불필요한 태그 제거
#             for tag in soup(['script', 'style']):
#                 tag.decompose()
                
#             # 본문 텍스트 추출    
#             text = soup.get_text()
#             lines = [line.strip() for line in text.splitlines() if line.strip()]
#             return '\n'.join(lines)
#     except Exception as e:
#         print(f"URL 접속 중 오류 발생: {url}")
#         print(f"오류 내용: {str(e)}")
#         return ""

def get_content_from_url(url: str) -> str:
    """URL에서 본문 내용을 추출합니다 (iframe 포함)"""
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    
    try:
        # Selenium 설정
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # 브라우저 창 숨기기
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        
        # iframe이 로드될 때까지 잠시 대기
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
        
        # 모든 iframe으로 전환하면서 내용 수집
        content = []
        
        # 메인 페이지 내용 추출
        main_content = driver.page_source
        soup = BeautifulSoup(main_content, 'html.parser')
        
        # 불필요한 태그 제거
        for tag in soup(['script', 'style']):
            tag.decompose()
            
        content.append(soup.get_text())
        
        # iframe 내용 추출
        iframes = driver.find_elements('tag name', 'iframe')
        for iframe in iframes:
            try:
                driver.switch_to.frame(iframe)
                iframe_content = driver.page_source
                iframe_soup = BeautifulSoup(iframe_content, 'html.parser')
                
                for tag in iframe_soup(['script', 'style']):
                    tag.decompose()
                    
                content.append(iframe_soup.get_text())
                driver.switch_to.default_content()
            except:
                continue
                
        driver.quit()
        
        # 전체 내용 정리
        text = '\n'.join(content)
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        
        # 텍스트 파일로 저장
        text_content = '\n'.join(lines)
        
        # URL에서 파일명 생성 (마지막 부분 추출)
        filename = url.split('/')[-1]
        if not filename.endswith('.html'):
            filename = f"{filename}.html"
            
        # 파일 저장
        with open(f"example/{filename}", 'w', encoding='utf-8') as f:
            f.write(text_content)
            
        print(f"파일 저장 완료: example/{filename}")
        return text_content
        
    except Exception as e:
        print(f"URL 접속 중 오류 발생: {url}")
        print(f"오류 내용: {str(e)}")
        return ""

# 검색 결과의 각 항목에서 링크 추출하고 내용 가져오기
contents = []
for item in results['items']:
    url = item['link']
    title = item['title']
    print(f"\n제목: {BeautifulSoup(title, 'html.parser').get_text()}")
    print(f"URL: {url}")
    
    content = get_content_from_url(url)
    if content:
        contents.append({
            'title': BeautifulSoup(title, 'html.parser').get_text(),
            'url': url,
            'content': content
        })
        print(f"내용 길이: {len(content)} 글자")
    else: 
        print("내용을 가져오지 못했습니다")

print(f"\n총 {len(contents)}개의 페이지에서 내용을 추출했습니다.")



제목: 가벼운 노트북 추천 13인치 가성비노트북 베이직북13 써보니
URL: https://blog.naver.com/dicagallery/222648318740
파일 저장 완료: example/222648318740.html
내용 길이: 10768 글자

제목: 노트북 화면 크기 추천! 14인치 15인치 16인치 비교, 리뷰, 추천크기, 무게. 나에게 맞는 노트북 크기는?
URL: https://blog.naver.com/yhd2613/223228258562
파일 저장 완료: example/223228258562.html
내용 길이: 10558 글자

제목: 삼성노트북 추천! 13인치 가성비 좋은 대학생노트북 삼성 노트북7 NT730XBV-A58A
URL: https://m.post.naver.com/viewer/postView.nhn?volumeNo=24419062&memberNo=1834
파일 저장 완료: example/postView.nhn?volumeNo=24419062&memberNo=1834.html
내용 길이: 5008 글자

제목: lg 울트라 노트북 13인치 가성비 노트북 추천 13UD50N-GX50K
URL: https://blog.naver.com/joytoseon/222153593503
파일 저장 완료: example/222153593503.html
내용 길이: 8281 글자

제목: 13인치 노트북 추천 삼성 노트북 Pen Active 1주일 사용기
URL: https://blog.naver.com/yeon5853/221466560408
파일 저장 완료: example/221466560408.html
내용 길이: 8393 글자

제목: 2024 LG전자 그램 노트북 추천 13인치 15인치 14인치 17인치 모델 비교 및 선택 가이드 LG그램 노트북 추천 사무....
URL: https://blog.naver.com/reji296758/223637739347
파일 저장 완료: example/223637739347.html
내용 길이: 